In [16]:
import math
import numpy as np
import pandas as pd
import csv

import psycopg2

from geographiclib.geodesic import Geodesic

import neo4j

from IPython.display import display
pd.set_option('display.max_rows', None)  # Show all rows


# Rank BART Stations by proximity to zipcodes and population size

In [2]:


# Load station information
df = pd.read_csv('stations.csv')

# Creating a list of station names
first_column_list = df.iloc[:, 0].tolist()

# Display the list
print(first_column_list)

['12th Street', '16th Street Mission', '19th Street', '24th Street Mission', 'Antioch', 'Ashby', 'Balboa Park', 'Bay Fair', 'Berryessa', 'Castro Valley', 'Civic Center', 'Coliseum', 'Colma', 'Concord', 'Daly City', 'Downtown Berkeley', 'Dublin', 'El Cerrito Plaza', 'El Cerrito del Norte', 'Embarcadero', 'Fremont', 'Fruitvale', 'Glen Park', 'Hayward', 'Lafayette', 'Lake Merritt', 'MacArthur', 'Millbrae', 'Milpitas', 'Montgomery Street', 'North Berkeley', 'North Concord', 'OAK', 'Orinda', 'Pittsburg', 'Pittsburg Center', 'Pleasant Hill', 'Powell Street', 'Richmond', 'Rockridge', 'SFO', 'San Bruno', 'San Leandro', 'South Hayward', 'South San Francisco', 'Union City', 'Walnut Creek', 'Warm Springs', 'West Dublin', 'West Oakland']


In [4]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [5]:
cursor = connection.cursor()

In [9]:
def my_calculate_box(point, miles):
    "Given a point and miles, calculate the box in form left, right, top, bottom"
    
    geod = Geodesic.WGS84

    kilometers = miles * 1.60934
    meters = kilometers * 1000

    g = geod.Direct(point[0], point[1], 270, meters)
    left = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 90, meters)
    right = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 0, meters)
    top = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 180, meters)
    bottom = (g['lat2'], g['lon2'])
    
    return(left, right, top, bottom)

In [10]:
def get_stations_ordered_by_zip_and_population():
    """Identify and order stations by the number of zip codes and population within each station's area."""

    # Query to get station data
    query_stations = "SELECT station, latitude, longitude FROM stations"
    cursor.execute(query_stations)
    stations = cursor.fetchall()
    
    if not stations:
        print("No stations found.")
        return

    # Dictionary to store results
    station_stats = {}

    for station, latitude, longitude in stations:
        # Calculate the boundary box for the desired radius (e.g., miles)
        point = (latitude, longitude)
        left, right, top, bottom = my_calculate_box(point, miles=3)  # Adjust the distance as needed

        # Query to get zip codes and populations within the area
        query_zips = """
            SELECT zip, population FROM zip_codes
            WHERE latitude >= %s AND latitude <= %s
              AND longitude >= %s AND longitude <= %s
        """
        cursor.execute(query_zips, (bottom[0], top[0], left[1], right[1]))
        zips = cursor.fetchall()
        
        # Calculate statistics
        num_zips = len(zips)
        total_population = sum(pop for _, pop in zips)

        # Store results
        station_stats[station] = {"num_zips": num_zips, "total_population": total_population}

    # Sort stations by number of zip codes and population
    sorted_stations = sorted(
        station_stats.items(),
        key=lambda x: (x[1]["num_zips"], x[1]["total_population"]),
        reverse=True
    )

    # Display the results
    print("\n-------------------------------------------------------------------------------")
    print("  Stations Ordered by Number of Zip Codes and Population")
    print("-------------------------------------------------------------------------------\n")
    for station, stats in sorted_stations:
        print(f"Station: {station}, Zip Codes: {stats['num_zips']}, Population: {stats['total_population']:10,}")
    print("\n-------------------------------------------------------------------------------")

# Call the function
get_stations_ordered_by_zip_and_population()


-------------------------------------------------------------------------------
  Stations Ordered by Number of Zip Codes and Population
-------------------------------------------------------------------------------

Station: 16th Street Mission, Zip Codes: 20, Population:    554,106
Station: Civic Center, Zip Codes: 19, Population:    500,384
Station: 24th Street Mission, Zip Codes: 18, Population:    621,661
Station: Powell Street, Zip Codes: 18, Population:    496,390
Station: Embarcadero, Zip Codes: 16, Population:    425,020
Station: Montgomery Street, Zip Codes: 16, Population:    425,020
Station: Glen Park, Zip Codes: 15, Population:    624,151
Station: Rockridge, Zip Codes: 15, Population:    317,026
Station: Downtown Berkeley, Zip Codes: 13, Population:    218,308
Station: North Berkeley, Zip Codes: 13, Population:    218,308
Station: Balboa Park, Zip Codes: 12, Population:    548,413
Station: MacArthur, Zip Codes: 12, Population:    271,301
Station: Ashby, Zip Codes: 12, Po

#  LOUVAIN MODULARITY
Group the stations into communities, and then within each community, decide which station to add a distribution / distribution point. 

In [11]:
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","ucb_mids_w205"))

In [12]:
session = driver.session(database="neo4j")

In [13]:
def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

In [14]:
query = "CALL gds.graph.drop('ds_graph', false) yield graphName"
session.run(query)

query = """

CALL gds.graph.project('ds_graph', 'Station', 'LINK', 
                      {relationshipProperties: 'weight'})
"""

session.run(query)

In [17]:
query = """

CALL gds.louvain.stream('ds_graph', {includeIntermediateCommunities: true})
YIELD nodeId, communityId, intermediateCommunityIds
RETURN gds.util.asNode(nodeId).name AS name, communityId as community, intermediateCommunityIds as intermediate_community
ORDER BY community, name ASC

"""

my_neo4j_run_query_pandas(query)


,name,community,intermediate_community
0,arrive 16th Street Mission,7,"[3, 7, 7]"
1,arrive 24th Street Mission,7,"[7, 7, 7]"
2,blue 16th Street Mission,7,"[3, 7, 7]"
3,blue 24th Street Mission,7,"[7, 7, 7]"
4,depart 16th Street Mission,7,"[3, 7, 7]"
5,depart 24th Street Mission,7,"[7, 7, 7]"
6,green 16th Street Mission,7,"[3, 7, 7]"
7,green 24th Street Mission,7,"[7, 7, 7]"
8,red 16th Street Mission,7,"[3, 7, 7]"
9,red 24th Street Mission,7,"[7, 7, 7]"
